In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-19011461-waterquality/train.csv
/kaggle/input/2021-ai-19011461-waterquality/test.csv
/kaggle/input/2021-ai-19011461-waterquality/sample_submit.csv


In [2]:
import torch
import random
random.seed(777)
torch.manual_seed(777)

In [3]:
train = pd.read_csv("/kaggle/input/2021-ai-19011461-waterquality/train.csv")
test = pd.read_csv("/kaggle/input/2021-ai-19011461-waterquality/test.csv")
submit = pd.read_csv("/kaggle/input/2021-ai-19011461-waterquality/sample_submit.csv")

print(train.info())
print(test.info())
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   생태구명          367 non-null    object 
 1   정점명           367 non-null    object 
 2   하수처리장         367 non-null    object 
 3   관측년도          367 non-null    int64  
 4   관측월           367 non-null    int64  
 5   일자            367 non-null    object 
 6   하수처리량(㎥/일)    263 non-null    object 
 7   BOD           367 non-null    float64
 8   COD           361 non-null    float64
 9   SS            367 non-null    float64
 10  T-N           367 non-null    float64
 11  T-P           367 non-null    float64
 12  날씨            367 non-null    object 
 13  수심            367 non-null    float64
 14  투명도(m)        367 non-null    float64
 15  수온(℃)표층       367 non-null    float64
 16  클로로필A(㎍/L)표층  367 non-null    float64
dtypes: float64(9), int64(2), object(6)
memory usage: 48.9+ KB
None
<class 'pa

,생태구명,정점명,하수처리장,관측년도,관측월,일자,하수처리량(㎥/일),BOD,COD,SS,T-N,T-P,날씨,수심,투명도(m),수온(℃)표층,클로로필A(㎍/L)표층
0,대한해협,통영3,통영하수처리장,2021,2,2021-02-21,NaN,4.2,NaN,2.9,15.200,1.100,맑음,6.0,5.0,9.11,4.30
1,대한해협,기장2,해운대사업소,2016,11,2016-11-03,"39,012",0.9,5.7,1.0,7.216,0.870,맑음,20.0,3.1,18.10,0.88
2,제주,표선1,남원처리장,2018,2,2018-02-26,NaN,2.8,9.8,1.9,7.750,1.153,구름조금,44.0,11.0,15.02,0.55
3,대한해협,낙동4,녹산사업소,2013,11,2013-11-02,"77,632",3.2,14.4,1.5,6.154,0.447,맑음,22.5,3.4,20.39,6.03
4,대한해협,부산10,중앙사업소,2019,2,2019-02-02,"92,330",4.5,8.3,4.6,11.211,0.753,맑음,14.0,5.2,14.04,2.06


In [4]:
# 예측해야 하는 컬럼 : 클로로필A(㎍/L)표층
y_train = train['클로로필A(㎍/L)표층']
X_train = train.drop(['일자', '하수처리량(㎥/일)', '클로로필A(㎍/L)표층'], axis = 1)
X_test = test.drop(['일자', '하수처리량(㎥/일)'], axis = 1)

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() 

cols = ['생태구명', '정점명', '하수처리장', '날씨']
for c in cols:
    X_train[c] = le.fit_transform(X_train[c])
    X_test[c] = le.transform(X_test[c])

In [6]:
print(X_train.info())
print(X_test.info())
X_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   생태구명     367 non-null    int64  
 1   정점명      367 non-null    int64  
 2   하수처리장    367 non-null    int64  
 3   관측년도     367 non-null    int64  
 4   관측월      367 non-null    int64  
 5   BOD      367 non-null    float64
 6   COD      361 non-null    float64
 7   SS       367 non-null    float64
 8   T-N      367 non-null    float64
 9   T-P      367 non-null    float64
 10  날씨       367 non-null    int64  
 11  수심       367 non-null    float64
 12  투명도(m)   367 non-null    float64
 13  수온(℃)표층  367 non-null    float64
dtypes: float64(8), int64(6)
memory usage: 40.3 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   생태구명     123 non-null    int64  
 1   정점명    

,생태구명,정점명,하수처리장,관측년도,관측월,BOD,COD,SS,T-N,T-P,날씨,수심,투명도(m),수온(℃)표층
0,0,16,17,2021,2,4.2,NaN,2.9,15.200,1.100,3,6.0,5.0,9.11
1,0,1,18,2016,11,0.9,5.7,1.0,7.216,0.870,3,20.0,3.1,18.10
2,1,17,4,2018,2,2.8,9.8,1.9,7.750,1.153,2,44.0,11.0,15.02
3,0,4,5,2013,11,3.2,14.4,1.5,6.154,0.447,3,22.5,3.4,20.39
4,0,8,16,2019,2,4.5,8.3,4.6,11.211,0.753,3,14.0,5.2,14.04


In [7]:
median = X_train['COD'].median()
X_train['COD'] = X_train['COD'].fillna(median)
X_test ['COD'] = X_test ['COD'].fillna(median)

X_train['관측년도'] = X_train['관측년도'] - 2011 # min(min(X_test['관측년도']))
X_train['관측월'] =(X_train['관측월']/3).astype('int') # 분기
X_test['관측년도'] = X_test['관측년도'] - 2011
X_test['관측월'] =(X_test['관측월']/3).astype('int')

X_train_tensor = torch.FloatTensor(X_train.to_numpy())
X_test_tensor = torch.FloatTensor(X_test.to_numpy())
y_train_tensor = torch.FloatTensor(y_train.to_numpy()).unsqueeze(1)

In [8]:
import torch.nn as nn
linear1 = nn.Linear(14, 64)
linear2 = nn.Linear(64, 32)
linear3 = nn.Linear(32,1)

nn.init.xavier_normal_(linear1.weight)
nn.init.xavier_normal_(linear2.weight)
nn.init.xavier_normal_(linear3.weight)

relu = nn.ReLU()
dropout = nn.Dropout(p=0.2)

model = torch.nn.Sequential(linear1, relu, dropout,
                             linear2, relu, dropout,
                             linear3)

In [9]:
lr = 1e-3
nb_epoch = 10000

optimizer = torch.optim.Adam(model.parameters(), lr = lr)
loss = nn.MSELoss()

for epoch in range (nb_epoch+1):
    model.train()
    h = model(X_train_tensor)
    cost = loss(h, y_train_tensor)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if(epoch%1000 == 0):
        print(epoch, cost.item())

0 25.049283981323242
1000 3.7800116539001465
2000 2.0099098682403564
3000 1.4681185483932495
4000 1.5803558826446533
5000 1.115512490272522
6000 1.0201594829559326
7000 0.8279753923416138
8000 0.7957602143287659
9000 0.9013664126396179
10000 0.826298177242279


In [10]:
with torch.no_grad():
    model.eval()
    y_pred = model(X_test_tensor).detach().numpy()

In [11]:
submit['클로로필A(㎍/L)표층'] = y_pred
submit.to_csv("submission.csv", mode = 'w', index = False)
submit

,id,클로로필A(㎍/L)표층
0,0,2.961222
1,1,1.364673
2,2,3.684328
3,3,2.277775
4,4,2.187966
...,...,...
118,118,1.279362
119,119,0.817248
120,120,1.519460
121,121,1.298009
